In [1]:
import pickle , matplotlib.pyplot as plt, torch, numpy as np, argparse, time, os, scipy
from sklearn import linear_model
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from src.TorchDSP.pbc_new import NonlienarFeatures, FoPBC
from src.TorchDSP.dataloader import  get_signals
from src.TorchSimulation.receiver import  BER

train_path = "data/train_data_afterCDCDSP.pkl"
test_path = "data/test_data_afterCDCDSP.pkl"

class Data(Dataset):
    def __init__(self, Nch=1, Rs=40, Pch=0, L=400, rho=1, index_type='reduce-1', trucation=[2000, -2000]):
        signal, truth, code = get_signals(train_path, Nch=Nch, Rs=Rs, Pch=[Pch],  device='cpu')
        self.f = NonlienarFeatures(Nmodes=1, rho=rho, L = L, index_type=index_type)
        self.index = self.f.index

        # features
        self.X0 = torch.squeeze(signal.val)[:, None][trucation]                          # [N]
        self.X1 = torch.squeeze(self.f(signal.val, signal.val, signal.val))[trucation]   # [N, p]
        self.X2 = (self.X0**2)[:,None] * self.X1                                         # [N, p]

        # label
        self.symb = torch.squeeze(truth.val)[truncation]                                 # [N] 
        self.Y = self.symb - self.X0                                                     # [N]     

    def __len__(self):
        return len(self.X0)
    
    def __getitem__(self, idx):
        return self.X0[idx], self.X1[idx], self.X2[idx], self.Y[idx]    



class Net(nn.Module):
    '''
    
    '''
    def __init__(self, p):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(p, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
train_path = "data/train_data_afterCDCDSP.pkl"
test_path = "data/test_data_afterCDCDSP.pkl"

train_signal, train_truth, train_z = get_signals(train_path, Nch=1, Rs=40, Pch=[test_p], device='cpu')
test_signal, test_truth, test_z = get_signals(test_path, Nch=1, Rs=40, Pch=[test_p], device='cpu')
    

f = NonlienarFeatures(Nmodes=1, rho=1, L = 400, index_type='reduce-1')

X_train = torch.cat([torch.squeeze(train_signal.val)[:, None], torch.squeeze(f(train_signal.val, train_signal.val, train_signal.val))], dim=1)
X_test  = torch.cat([torch.squeeze(test_signal.val)[:, None], torch.squeeze(f(test_signal.val, test_signal.val, test_signal.val))], dim=1)
